In [1]:
import numpy as np
import sympy
import scipy.optimize as opt
import copy
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse import eye

In [2]:
# Parameters
rho = 0.05
r = 0.05
delta = 0.07
c_param = 2
l_param = 2
B_param = 2

params = [rho, r, delta, c_param, l_param]

# Time parameters
T = 70 # length of lifespan
nT = 280 # number of time grids
dt = T/nT # Length of each time grid

In [3]:
# Define utility functions
def u(c, c_param):
    return (c**(1-c_param))/(1-c_param)

def u_prime(c, c_param):
    return c**(-c_param)

def u_prime_inverse(x, c_param):
    return x**(-1/c_param)

def uh(h, l_param):
    return ((1-h)**(1-l_param))/(1-l_param)

def uh_prime(h, l_param):
    return -(1-h)**(-l_param)

def uh_prime_inverse(x, l_param):
    return 1 - ((-x)**(-1/l_param))

In [4]:
# Labor market functions
def g(x):
    return 1.25 - (x*(1.25**0.5 - 0.5) + 0.5)**2

def g_prime(x):
    return -2*(x*(1.25**0.5 - 0.5) + 0.5)*(1.25**0.5 - 0.5)

def g_prime_inverse(x):
    return (x/(-2*(1.25**0.5 - 0.5)) - 0.5)/(1.25**0.5 - 0.5)

Create grids

In [5]:
# State grids
na = 50 # number of asset grid points
nk = 50 # number of human capital grid points

amin = 1
amax = 10

kmin = 1
kmax = 10

a_vect = amin + np.linspace(0, 1, na)*(amax - amin) # Vector of A
da = (amax - amin)/(na - 1) # delta in A
aa = np.repeat(a_vect[:, None], nk, axis = 1) # Grid of A

k_vect = kmin + np.linspace(0, 1, nk)*(kmax - kmin) # Vector of A
dk = (kmax - kmin)/(nk - 1) # delta in K
kk = np.repeat(k_vect[None, :], na, axis = 0) # Grid of K

# Preallocate fwd/bwd derivatives, policy grid, value function grid
dVaF = np.zeros((na, nk))
dVaB = np.zeros((na, nk))
dVkF = np.zeros((na, nk))
dVkB = np.zeros((na, nk))

V_t = np.zeros((na, nk, nT+1))
C_t = np.zeros((na, nk, nT))
H_t = np.zeros((na, nk, nT))
X_t = np.zeros((na, nk, nT))
U_t = np.zeros((na, nk, nT))
mu_a_t = np.zeros((na, nk, nT))
mu_k_t = np.zeros((na, nk, nT))
dVa_t = np.zeros((na, nk, nT))
dVk_t = np.zeros((na, nk, nT))


# Arrays to store intermediate results for debugging
dVaF_t = np.zeros((na, nk, nT))
dVaB_t = np.zeros((na, nk, nT))
dVkF_t = np.zeros((na, nk, nT))
dVkB_t = np.zeros((na, nk, nT))
I_valid_t = np.zeros((na, nk, nT))
I_type_t = np.zeros((na, nk, nT))


# Terminal value
small_number1 = 10**(-16)
small_number2 = 10**(-16)
v_terminal = small_number1*((small_number2 + aa)**(1-B_param))/(1-B_param)

# Place terminal value into value function grid
V_t[:, :, nT] = v_terminal

Define functions that find dVa and dVk at boundaries a_min and k_min

In [6]:
# Drift functions
def drift_a(dVa, dVk, a, k, params):
    rho, r, delta, c_param, l_param = params
    c = u_prime_inverse(dVa, c_param)
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_a = r*a + g(x)*h*k - c
    return mu_a

def drift_k(dVk, dVa, a, k, params):
    rho, r, delta, c_param, l_param = params
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_k = a*x*h*k - delta*k
    return mu_k

In [7]:
def boundary_dVa(dVk, a, k, params):
    
    # Find dVa that leads to negative drift in a
    dVa_neg = 1
    while True:
        if drift_a(dVa_neg, dVk, a, k, params) < 0:
            break
        else:
            dVa_neg = dVa_neg/2

    # Find dVa that leads to positive drift in a
    dVa_pos = 1
    while True:
        if drift_a(dVa_pos, dVk, a, k, params) > 0:
            break
        else:
            dVa_pos = dVa_pos + 1

    # Use brentq to find the value of dVa where drift in a is 0, should be in between
    sol = opt.brentq(drift_a, dVa_neg, dVa_pos, args = (dVk, a, k, params), xtol = 1e-16, rtol = 1e-15)
    return sol

def boundary_dVk(dVa, a, k, params):
    
    # Find dVk that leads to negative drift in k
    dVk_neg = 1
    while True:
        if drift_k(dVk_neg, dVa, a, k, params) < 0:
            break
        else:
            dVk_neg = dVk_neg/2

    # Find dVk that leads to positive drift in k
    dVk_pos = 1
    while True:
        if drift_k(dVk_pos, dVa, a, k, params) > 0:
            break
        else:
            dVk_pos = dVk_pos + 1

    sol = opt.brentq(drift_k, dVk_neg, dVk_pos, args = (dVa, a, k, params), xtol = 1e-16, rtol = 1e-15)
    return sol

Find steady-state

In [8]:
# Define a function that computes mu_k for a given dVk where dVa is set to be such that mu_a=0
def drift_k_dVk(dVk, a, k, params):
    rho, r, delta, c_param, l_param = params

    # dVa that leads to mu_a = 0 for given dVk
    dVa = boundary_dVa(dVk, a, k, params)

    mu_k = drift_k(dVk, dVa, a, k, params)
    
    return mu_k

In [9]:
# Define a function that finds dVk that leads to mu_k=0 where dVa is set to be such that mu_a=0
def ss_dVk(a, k, params):
    rho, r, delta, c_param, l_param = params

    # Find dVk that leads to negative drift in k
    dVk_neg = 1
    while True:
        if drift_k_dVk(dVk_neg, a, k, params) < 0:
            break
        else:
            dVk_neg = dVk_neg/2

    # Find dVk that leads to positive drift in k
    dVk_pos = 1
    while True:
        if drift_k_dVk(dVk_pos, a, k, params) > 0:
            break
        else:
            dVk_pos = dVk_pos + 1

    sol = opt.brentq(drift_k_dVk, dVk_neg, dVk_pos, args = (a, k, params), xtol = 1e-16, rtol = 1e-15)

    return sol

In [10]:
# Define a function that computes controls and drifts for given dV and state
def control_drift(dVa, dVk, a, k, params):
    rho, r, delta, c_param, l_param = params

    c = u_prime_inverse(dVa, c_param)
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_a = r*a + g(x)*h*k - c
    mu_k = a*x*h*k - delta*k

    return c, h, x, mu_a, mu_k

In [11]:
# Find steady-state dVk0
dVk0 = np.zeros((na, nk))
dVa0 = np.zeros((na, nk))

for i in range(na):
    for j in range(nk):
        dVk0[i,j] = ss_dVk(aa[i,j], kk[i,j], params)
        dVa0[i,j] = boundary_dVa(dVk0[i,j], aa[i,j], kk[i,j], params)

In [12]:
# Fill in the rest of the steady-state values
C0, H0, X0, mu_a0, mu_k0 = control_drift(dVa0, dVk0, aa, kk, params)

NameError: name &#39;dVa&#39; is not defined

Begin iterations

In [207]:
for t in range(nT-1, -1, -1):
    age = t*dt
    print(f'Age is {age}.')
    V = V_t[:, :, t+1]

    # Construct forward and backward differences for a and k
    dVaF[0:na-1, :] = (V[1:na, :] - V[0:na-1, :])/da
    dVaF[na-1, :] = dVaF[na-2, :]

    dVkF[:, 0:nk-1] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
    dVkF[:, nk-1] = dVkF[:, nk-2]

    dVaB[1:na, :] = (V[1:na, :] - V[0:na-1, :])/da
    dVaB[0, :] = dVaB[1, :]

    dVkB[:, 1:nk] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
    dVkB[:, 0] = dVkB[:, 1]

    # Store differences for debugging
    dVaF_t[:, :, t] = copy.copy(dVaF)
    dVaB_t[:, :, t] = copy.copy(dVaB)
    dVkF_t[:, :, t] = copy.copy(dVkF)
    dVkB_t[:, :, t] = copy.copy(dVkB)

    # Verify value function is increasing in a
    if np.sum(dVaF < 0) > 0 or np.sum(dVaB < 0) > 0: # Print message if there exists an element of dVa < 0
        print('V is not monotonically increasing in a')

    if np.sum(dVkF < 0) > 0 or np.sum(dVkB < 0) > 0: # Print message if there exists an element of dVk < 0
        print('V is not monotonically increasing in k')

    # Compute policies and drift
    cFF, hFF, xFF, mu_aFF, mu_kFF = control_drift(dVaF, dVkF, aa, kk, params)
    cFB, hFB, xFB, mu_aFB, mu_kFB = control_drift(dVaF, dVkB, aa, kk, params)
    cBF, hBF, xBF, mu_aBF, mu_kBF = control_drift(dVaB, dVkF, aa, kk, params)
    cBB, hBB, xBB, mu_aBB, mu_kBB = control_drift(dVaB, dVkB, aa, kk, params)

    # Create indicator matrices indicating consistency of direction of drift with fwd/bwd difference
    # Impose False at boundary if fwd/bwd difference implies drift sends state beyond boundary
    I_FF = np.logical_and(mu_aFF > 0, mu_kFF > 0)
    I_FF[na-1, :] = False
    I_FF[:, nk-1] = False

    I_FB = np.logical_and(mu_aFB > 0, mu_kFB < 0)
    I_FB[na-1, :] = False
    I_FB[:, 0] = False

    I_BF = np.logical_and(mu_aBF < 0, mu_kBF > 0)
    I_BF[0, :] = False
    I_BF[:, nk-1] = False

    I_BB = np.logical_and(mu_aBB < 0, mu_kBB < 0)
    I_BB[0, :] = False
    I_BB[:, 0] = False

    # Compute validity of fwd/bwd difference combinations
    # Stack all indicator matrices
    I_stacked = np.zeros((na, nk, 4))
    I_stacked[:, :, 0] = I_FF
    I_stacked[:, :, 1] = I_FB
    I_stacked[:, :, 2] = I_BF
    I_stacked[:, :, 3] = I_BB

    # Use logical-or over all 4 combinations; if at least one is valid, I_valid is True
    I_valid = np.logical_or.reduce((I_FF, I_FB, I_BF, I_BB))
    I_type = np.argmax(I_stacked, axis = 2)

    # Impose state boundary condition for a_min
    # Loop over a_min (upper edge) in the j dimension
    aind = 0
    aval = aa[aind, 0]
    for j in range(nk):
        if I_valid[aind, j] == False:
            kj = kk[aind, j]

            # Try BF scheme, so use dVkF
            dVkj = dVkF[aind, j]
            dVaj = boundary_dVa(dVkj, aval, kj, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
            cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

            # Check consistency of upwind scheme
            if mu_kj>=0 and j!=nk-1:
                # fwd diff for k is successful; BF scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 2

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aBF[aind, j] = mu_aj
                mu_kBF[aind, j] = mu_kj

            # BF scheme failed; try BB scheme
            else:
                dVkj = dVkB[aind, j]
                dVaj = boundary_dVa(dVkj, aval, kj, params)

                cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

                if mu_kj <= 0 and j != 0:
                    # bwd diff for k is successful; BB scheme works
                    I_valid[aind, j] = True
                    I_type[aind, j] = 3

                    # Replace bwd diff approximation for aval at j
                    dVaB[aind, j] = dVaj

                    # Replace bwd diff drift values
                    mu_aBB[aind, j] = mu_aj
                    mu_kBB[aind, j] = mu_kj
    print('a_min completed')

    # Impose state boundary condition for a_max
    # First look at the case with mu_kFF
    # Loop over a_max (lower edge) in the j dimension
    aind = na-1
    aval = aa[aind, 0]
    for j in range(nk):
        if I_valid[aind, j] == False:
            kj = kk[aind, j]

            # Try FF scheme
            dVkj = dVkF[aind, j]

            # Find dVaF that satisfies FOCs and a_max boundary condition
            dVa_guess = dVaF[aind, j]
            dVaj = boundary_dVa(dVkj, aval, kj, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
            cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

            if mu_kj>=0 and j!=nk-1:
                # fwd diff for k is successful; FF scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 0

                # Replace fwd diff approximation for aval at j
                dVaF[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aFF[aind, j] = mu_aj
                mu_kFF[aind, j] = mu_kj

            # FF scheme failed; try FB scheme
            else:
                dVkj = dVkB[aind, j]
                dVaj = boundary_dVa(dVkj, aval, kj, params)

                cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

                if mu_kj<=0 and j!=0:
                    # bwd diff for k is successful; BB scheme works
                    I_valid[aind, j] = True
                    I_type[aind, j] = 1

                    # Replace bwd diff approximation for aval at j
                    dVaB[aind, j] = dVaj

                    # Replace bwd diff drift values
                    mu_aFB[aind, j] = mu_aj
                    mu_kFB[aind, j] = mu_kj
    
    print('a_max completed')

    # Impose state boundary condition for k_min
    # Loop over k_min (left edge) in the i dimension
    kind = 0
    kval = kk[0, kind]
    for i in range(na):
        if I_valid[i, kind] == False:
            ai = aa[i, kind]

            # Try FB scheme first
            dVai = dVaF[i, kind]

            # Find dVkB that satisfies FOC and k_min boundary condition
            dVki = boundary_dVk(dVai, ai, kval, params)

            cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

            if mu_ai>=0 and i!=na:
                # fwd diff for a is successful; FB scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 1

                # Replace bwd diff approximation for kmin at i
                dVkB[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aFB[i, kind] = mu_ai
                mu_kFB[i, kind] = mu_ki

            # FB scheme failed, try BB
            else:
                dVai = dVaB[i, kind]

                # Find dVkB that satisfies FOC and k_min boundary condition
                dVki = boundary_dVk(dVai, ai, kval, params)

                # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
                cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

                if mu_ai<=0 and i!=0:
                    # bwd diff for a is successful; BB scheme works
                    I_valid[i, kind] = True
                    I_type[i, kind] = 3

                    # Replace bwd diff approximation for amax at j
                    dVkB[i, kind] = dVki

                    # Replace bwd diff drift values
                    mu_aBB[i, kind] = mu_ai
                    mu_kBB[i, kind] = mu_ki

    print('k_min completed')

    # Impose state boundary condition for k_max
    # First look at the case with mu_aFF
    # Loop over k_max (upper edge) in the i dimension
    kind = nk-1
    kval = kk[0, kind]
    for i in range(na):
        if I_valid[i, kind] == False:
            ai = aa[i, kind]
            dVai = dVaF[i, kind]

            # Find dVkF that satisfies FOC and k_max boundary condition
            dVki = boundary_dVk(dVai, ai, kval, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

            if mu_ai>=0 and i!=na:
                # fwd diff for a is successful; FF scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 0

                # Replace fwd diff approximation for kmax at i
                dVkF[i, kind] = dVki

                # Replace drift values
                mu_aFF[i, kind] = mu_ai
                mu_kFF[i, kind] = mu_ki

            # FF scheme failed, try BF
            else:
                dVai = dVaB[i, kind]

                # Find dVkF that satisfies FOC and k_min boundary condition
                dVki = boundary_dVk(dVai, ai, kval, params)

                # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
                cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

                if mu_ai<=0 and i!=0:
                    # bwd diff for a is successful; BF scheme works
                    I_valid[i, kind] = True
                    I_type[i, kind] = 2

                    # Replace fwd diff approximation for kmax at i
                    dVkF[i, kind] = dVki

                    # Replace bwd diff drift values
                    mu_aBF[i, kind] = mu_ai
                    mu_kBF[i, kind] = mu_ki

    print('k_max completed')

    # Construct dVa and dVk matrices
    # If no scheme is valid, use steady-state values, i.e. stay put
    # Note: FF is I_type == 0, FB is 2, BF is 3, BB is 4
    dVa = (dVaF*(I_type == 0) + dVaF*(I_type == 1) + dVaB*(I_type == 2) + dVaB*(I_type == 3))*I_valid + dVa0*(~I_valid)
    dVk = (dVkF*(I_type == 0) + dVkB*(I_type == 1) + dVkF*(I_type == 2) + dVkB*(I_type == 3))*I_valid + dVk0*(~I_valid)

    # Construct policy functions and utility
    C, H, X, mu_a_test, mu_k_test = control_drift(dVa, dVk, aa, kk, params)
    U = u(C, c_param) + uh(H, l_param)

    # Construct drift values
    mu_a = (mu_aFF*(I_type == 0) + mu_aFB*(I_type == 1) + mu_aBF*(I_type == 2) + mu_aBB*(I_type == 3))*I_valid
    mu_k = (mu_kFF*(I_type == 0) + mu_kFB*(I_type == 1) + mu_kBF*(I_type == 2) + mu_kBB*(I_type == 3))*I_valid

    # Construct sparse diagonal matrix A
    # Reshape mu_a using column-major (Fortran) order
    mu_a_diag = np.reshape(mu_a, na*nk, order='F')/da

    # -1 diagonal is mu_a<0, +1 diag is mu>0
    # Shift elements forward 1 index as placement in upper diagonal in spdiags lops off the first element. Want diagonal to start with the 1st element.
    mu_a_diag_pos = np.roll(np.clip(mu_a_diag, 0, None), 1)

    # Shift elements back 1 index as placement in lower diagonal in spdiags lops off the last element. Want diagonal to start with the 2nd element.
    mu_a_diag_neg = np.roll(-np.clip(mu_a_diag, None, 0), -1) 

    # Reshape mu_k using column-major (Fortran) order
    mu_k_diag = np.reshape(mu_k, na*nk, order='F')/dk

    # -na diag is mu_k < 0, +na diag is mu_k > 0
    # Shift elements forward na index as placement in upper diagonal in spdiags lops off the first na elements. Want diagonal to start with 1st element.
    mu_k_diag_pos = np.roll(np.clip(mu_k_diag, 0, None), na)

    # Shift elements back na index as placement in lower diagonal in spdiags lops off the last na elements. Want diagonal to start with the na'th element
    mu_k_diag_neg = np.roll(-np.clip(mu_k_diag, None, 0), -na)

    # Construct main diagonal
    main_diag = np.reshape(-np.abs(mu_a)/da - np.abs(mu_k)/dk, na*nk, order='F')

    A = spdiags(np.array([mu_k_diag_neg, mu_a_diag_neg, main_diag, mu_a_diag_pos, mu_k_diag_pos]), np.array([-na, -1, 0, 1, na]), na*nk, na*nk)

    # Solve set of linear equations to obtain V for current period from V of next period
    b = np.reshape(U + V/dt, na*nk, order='F')
    B = (1/dt + rho)*np.identity(na*nk) - A
    V_stacked = np.linalg.solve(B, b)
    
    V_t[:, :, t] = copy.copy(np.reshape(V_stacked, (na, nk), order = 'F'))
    C_t[:, :, t] = copy.copy(C)
    H_t[:, :, t] = copy.copy(H)
    X_t[:, :, t] = copy.copy(X)
    U_t[:, :, t] = copy.copy(U)

    mu_a_t[:, :, t] = copy.copy(mu_a)
    mu_k_t[:, :, t] = copy.copy(mu_k)
    dVa_t[:, :, t] = copy.copy(dVa)
    dVk_t[:, :, t] = copy.copy(dVk)
    I_valid_t[:, :, t] = copy.copy(I_valid)
    I_type_t[:, :, t] = copy.copy(I_type)

Age is 69.75.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 69.5.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 69.25.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 69.0.
V is not monotonically increasing in k
a_min completed
a_max completed
k_min completed
k_max completed
Age is 68.75.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 68.5.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 68.25.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 68.0.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 67.75.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 67.5.
V is not monotonically increasing in a
a_min completed
a_max completed
k_min completed
k_max completed
Age is 67.25.


ZeroDivisionError: 0.0 cannot be raised to a negative power

In [208]:
np.min(dVaF_t[:, :, nT-11])

nan

In [209]:
np.sum(dVaF_t[:, :, nT-11] < 0, axis = 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [210]:
dVaF_t[6, 3, nT-11]

nan

In [211]:
I_valid_t[:, :, nT-4]

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1.

In [212]:
I_type_t[:, 1, nT-4]

array([3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [213]:
dVkB_t[:, 1, nT-4]

array([1.07408286, 1.05188225, 1.03283351, 1.01661862, 1.00050613,
       0.98515347, 0.97075481, 0.95733549, 0.94485661, 0.93325616,
       0.9224662 , 0.91242018, 0.90305587, 0.89431634, 0.88615013,
       0.87851088, 0.871357  , 0.86465107, 0.8583595 , 1.52076488])

In [214]:
dVaF = dVaF_t[:, :, nT-4]
dVaB = dVaB_t[:, :, nT-4]
dVkF = dVkF_t[:, :, nT-4]
dVkB = dVkB_t[:, :, nT-4]

In [215]:
# Compute policies and drift
cFF, hFF, xFF, mu_aFF, mu_kFF = control_drift(dVaF, dVkF, aa, kk, params)
cFB, hFB, xFB, mu_aFB, mu_kFB = control_drift(dVaF, dVkB, aa, kk, params)
cBF, hBF, xBF, mu_aBF, mu_kBF = control_drift(dVaB, dVkF, aa, kk, params)
cBB, hBB, xBB, mu_aBB, mu_kBB = control_drift(dVaB, dVkB, aa, kk, params)

In [222]:
mu_kBB

array([[-0.03489733,  0.19918538, -0.03963916, -0.16947368, -0.20263158,
        -0.23578947, -0.26894737, -0.30210526, -0.33526316, -0.36842105,
        -0.40157895, -0.43473684, -0.46789474, -0.50105263, -0.53421053,
        -0.56736842, -0.60052632, -0.63368421, -0.66684211, -0.7       ],
       [-0.033197  ,  0.32427962, -0.01869587, -0.16947368, -0.20263158,
        -0.23578947, -0.26894737, -0.30210526, -0.33526316, -0.36842105,
        -0.40157895, -0.43473684, -0.46789474, -0.50105263, -0.53421053,
        -0.56736842, -0.60052632, -0.63368421, -0.66684211, -0.7       ],
       [-0.03879684,  0.44051179, -0.01341497, -0.16947368, -0.20263158,
        -0.23578947, -0.26894737, -0.30210526, -0.33526316, -0.36842105,
        -0.40157895, -0.43473684, -0.46789474, -0.50105263, -0.53421053,
        -0.56736842, -0.60052632, -0.63368421, -0.66684211, -0.7       ],
       [-0.05013004,  0.54978394, -0.02577068, -0.16947368, -0.20263158,
        -0.23578947, -0.26894737, -0.30210526, -